<a href="https://colab.research.google.com/github/Lilwm/Augmenting-Pandas-with-SQLite/blob/main/Augmenting_Pandas_with_SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Notebook: Augmenting Pandas with SQLite

## Question 1: Introduction

In this session, we explored a few different ways to work with larger datasets in pandas. In this guided project, we'll practice using some of the techniques we learned to analyze startup investments from Crunchbase.com.

Every year, thousands of startup companies raise financing from investors. Each time a startup raises money, we refer to the event as a fundraising round. Crunchbase is a website that crowdsources information on the fundraising rounds of many startups. The Crunchbase user community submits, edits, and maintains most of the information in Crunchbase.

In return, Crunchbase makes the data available through a Web application and a fee-based API. Before Crunchbase switched to the paid API model, multiple groups crawled the site and released the data online. Because the information on the startups and their fundraising rounds is always changing, the data set we'll be using isn't completely up to date.

Throughout this project, we'll practice working with different memory constraints. In this step, let's assume we only have 10 megabytes of available memory. While crunchbase-investments.csv (https://bit.ly/3BPcobU) consumes 10.3 megabytes of disk space, we know from earlier lessons that pandas often requires 4 to 6 times amount of space in memory as the file does on disk (especially when there's many string columns).


**Tasks**

* Because the data set contains over 50,000 rows, you'll need to read the data set into dataframes using 5,000 row chunks to ensure that each chunk consumes much less than 10 megabytes of memory.
* Across all of the chunks, become familiar with:
1. Each column's missing value counts.
2. Each column's memory footprint.
3. The total memory footprint of all of the chunks combined.
4. Which column(s) we can drop because they aren't useful for analysis.



In [100]:
# Dataset URL = https://bit.ly/3BPcobU

#import pandas
import pandas as pd

#explore the data
df = pd.read_csv('https://bit.ly/3BPcobU', encoding='latin1', low_memory=False)
df.head(5)

,company_permalink,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_category_code,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
0,/company/advercar,AdverCar,advertising,USA,CA,SF Bay,San Francisco,/company/1-800-flowers-com,1-800-FLOWERS.COM,NaN,USA,NY,New York,New York,series-a,2012-10-30,2012-10,2012-Q4,2012.0,2000000.0
1,/company/launchgram,LaunchGram,news,USA,CA,SF Bay,Mountain View,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-23,2012-01,2012-Q1,2012.0,20000.0
2,/company/utap,uTaP,messaging,USA,NaN,United States - Other,NaN,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-01,2012-01,2012-Q1,2012.0,20000.0
3,/company/zoopshop,ZoopShop,software,USA,OH,Columbus,columbus,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,angel,2012-02-15,2012-02,2012-Q1,2012.0,20000.0
4,/company/efuneral,eFuneral,web,USA,OH,Cleveland,Cleveland,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2011-09-08,2011-09,2011-Q3,2011.0,20000.0


In [101]:
# check each column's missing values
missing = df.isnull().sum()
#missing value %
print(f'Missing values in percent \n{round(100 * missing/df.shape[0], 2)}')

Missing values in percent 
company_permalink          0.00
company_name               0.00
company_category_code      1.22
company_country_code       0.00
company_state_code         0.93
company_region             0.00
company_city               1.01
investor_permalink         0.00
investor_name              0.00
investor_category_code    95.38
investor_country_code     22.70
investor_state_code       31.79
investor_region            0.00
investor_city             23.61
funding_round_type         0.01
funded_at                  0.01
funded_month               0.01
funded_quarter             0.01
funded_year                0.01
raised_amount_usd          6.81
dtype: float64


In [139]:
# Each column's missing value counts.
chunk_iter = pd.read_csv('https://bit.ly/3BPcobU', chunksize=5000, encoding='latin1',)

chunk_idx = 0
missing_values = []
for chunk in chunk_iter:    
    missing_value_counts = chunk.isnull().sum()
    missing_values.append(missing_value_counts)
    chunk_idx += 1
    print(f'Chunk {chunk_idx}\nEach column missing value count is: \n{missing_value_counts}\n')

total_missing_value = pd.concat(missing_values)
#group up the columns missing values
total_missing_value = total_missing_value.groupby(total_missing_value.index).mean()
#columns with alot of missing value that can be dropped i.e with 
most_missing_cols = list(total_missing_value[total_missing_value > 95].index)
few_missing_cols = list(total_missing_value[(total_missing_value > 25) & (total_missing_value <95) ].index)

Chunk 1
Each column missing value count is: 
company_permalink            0
company_name                 0
company_category_code       52
company_country_code         0
company_state_code          53
company_region               0
company_city                64
investor_permalink           0
investor_name                0
investor_category_code    2557
investor_country_code      778
investor_state_code       1371
investor_region              0
investor_city              900
funding_round_type           0
funded_at                    0
funded_month                 0
funded_quarter               0
funded_year                  0
raised_amount_usd          653
dtype: int64

Chunk 2
Each column missing value count is: 
company_permalink            0
company_name                 0
company_category_code       51
company_country_code         0
company_state_code          43
company_region               0
company_city                45
investor_permalink           0
investor_name               

In [103]:
# memory footprint

crunch_iter = pd.read_csv('https://bit.ly/3BPcobU', encoding = "latin1", chunksize = 5000,)

chunk_idx = 0
chunk_memory_usage = 0
initial_memory_usage = 0

for chunk in crunch_iter:
  chunk_memory_usage = chunk.memory_usage(deep=True).sum()/(1024**2)
  initial_memory_usage +=chunk_memory_usage
  chunk_idx += 1
  print(f'Chunk {chunk_idx} Memory Usage: {chunk_memory_usage}MB')

print(f'\ntotal memory footprint of all chunks combined is:{initial_memory_usage}MB')


Chunk 1 Memory Usage: 5.579240798950195MB
Chunk 2 Memory Usage: 5.528232574462891MB
Chunk 3 Memory Usage: 5.535050392150879MB
Chunk 4 Memory Usage: 5.528193473815918MB
Chunk 5 Memory Usage: 5.5243377685546875MB
Chunk 6 Memory Usage: 5.553427696228027MB
Chunk 7 Memory Usage: 5.531436920166016MB
Chunk 8 Memory Usage: 5.5096588134765625MB
Chunk 9 Memory Usage: 5.396121025085449MB
Chunk 10 Memory Usage: 4.639497756958008MB
Chunk 11 Memory Usage: 2.6637144088745117MB

total memory footprint of all chunks combined is:56.988911628723145MB


***Unnecessary columns***

* We can drop the columns `funded_month` and` funded_year` because they contain redundant info that is already contained in `funded_at` column

* we can also drop `investor_category_code ` column because over 95% of the values are missing

## Question 2: Selecting Data Types

Now that we have a good sense of the missing values, let's get familiar with the column types before adding the data into SQLite.

**Tasks**

* Identify the types for each column.
* Identify the numeric columns we can represent using more space efficient types.
For text columns:
* Analyze the unique value counts across all of the chunks to see if we can convert them to a numeric type.
* See if we clean clean any text columns and separate them into multiple numeric columns without adding any overhead when querying.
* Make your changes to the code from the last step so that the overall memory the data consumes stays under 10 megabytes.


In [104]:
#Identify the types for each column.

use_cols = ['investor_region', 'investor_permalink', 'investor_name',
        'investor_country_code', 'investor_city', 'funding_round_type',
        'company_category_code', 'funded_at', 'company_state_code',
        'company_region', 'company_permalink', 'company_name', 
        'company_country_code', 'company_city', 'investor_state_code', 
        'funded_quarter', 'raised_amount_usd']
df = pd.read_csv( 'https://bit.ly/3BPcobU', encoding = "latin1",  usecols=use_cols)
df.dtypes

company_permalink         object
company_name              object
company_category_code     object
company_country_code      object
company_state_code        object
company_region            object
company_city              object
investor_permalink        object
investor_name             object
investor_country_code     object
investor_state_code       object
investor_region           object
investor_city             object
funding_round_type        object
funded_at                 object
funded_quarter            object
raised_amount_usd        float64
dtype: object

*From the results above, we only have one numeric column `raised_amount_usd`. let's optimize memory from a float64 dtyp*e 

In [105]:
# Identify the numeric columns we can represent using more space efficient types
import numpy as np
#function to convert to int for cols with no missing values
def change_to_int(df, col_name):
    # Get the min and max values
    max_col = df[col_name].max()
    min_col = df[col_name].min()
    # Find the datatype
    for dtype_name in ['int8', 'int16', 'int32', 'int64']:
        # Check if this datatype can hold all values
        if max_col <  np.iinfo(dtype_name).max and min_col > np.iinfo(dtype_name).min:
            df[col_name] = df[col_name].astype(dtype_name)
            break

use_cols = ['investor_region', 'investor_permalink', 'investor_name',
        'investor_country_code', 'investor_city', 'funding_round_type',
        'company_category_code', 'funded_at', 'company_state_code',
        'company_region', 'company_permalink', 'company_name', 
        'company_country_code', 'company_city', 'investor_state_code', 
        'funded_quarter', 'raised_amount_usd']

chunk_iter = pd.read_csv( 'https://bit.ly/3BPcobU', encoding = "latin1",  usecols=use_cols, chunksize=5000)
Total_memory = 0
for chunk in chunk_iter:
  chunk['raised_amount_usd'] = chunk['raised_amount_usd'].fillna(0)
  change_to_int(chunk, 'raised_amount_usd' )
  chunk_memory = chunk.memory_usage(deep=True).sum() /(1024**2)
  Total_memory +=chunk_memory

  print(chunk.dtypes)
  print('==' *20)

print(f'Total Memory {Total_memory}MB')



company_permalink        object
company_name             object
company_category_code    object
company_country_code     object
company_state_code       object
company_region           object
company_city             object
investor_permalink       object
investor_name            object
investor_country_code    object
investor_state_code      object
investor_region          object
investor_city            object
funding_round_type       object
funded_at                object
funded_quarter           object
raised_amount_usd         int64
dtype: object
company_permalink        object
company_name             object
company_category_code    object
company_country_code     object
company_state_code       object
company_region           object
company_city             object
investor_permalink       object
investor_name            object
investor_country_code    object
investor_state_code      object
investor_region          object
investor_city            object
funding_round_type       o

*We have saved ~ 4MB by converting `raised_amount_usd ` column from float64 to int32*



In [106]:
#optimize string cols 
total_memory = 0
chunk_iter = pd.read_csv( 'https://bit.ly/3BPcobU', encoding = "latin1",  usecols=use_cols, chunksize=5000)
for chunk in chunk_iter:
    # Optimize numeric column
    chunk['raised_amount_usd'] = chunk['raised_amount_usd'].fillna(0)
    change_to_int(chunk, 'raised_amount_usd' )

    # Optimize 'funded_quarter' column
    chunk['funded_quarter'] = chunk['funded_quarter'].str.extract(r'Q(\d)', expand=False)
    chunk['funded_quarter'] = chunk['funded_quarter'].fillna(-1)
    chunk['funded_quarter'] = chunk['funded_quarter'].astype('int8')

    chunk_memory = chunk.memory_usage(deep=True).sum()
    total_memory += chunk_memory
    print('Chunk memory usage (MB) =', chunk_memory/(1024**2))
print('--'*20)
print('Total memory usage (MB) =', total_memory/(1024**2))
print('--'*20)
print(chunk.dtypes)

Chunk memory usage (MB) = 4.70713996887207
Chunk memory usage (MB) = 4.827281951904297
Chunk memory usage (MB) = 4.834099769592285
Chunk memory usage (MB) = 4.827242851257324
Chunk memory usage (MB) = 4.823387145996094
Chunk memory usage (MB) = 4.871550559997559
Chunk memory usage (MB) = 4.830669403076172
Chunk memory usage (MB) = 4.808708190917969
Chunk memory usage (MB) = 4.6951704025268555
Chunk memory usage (MB) = 3.938547134399414
Chunk memory usage (MB) = 2.261368751525879
----------------------------------------
Total memory usage (MB) = 49.42516613006592
----------------------------------------
company_permalink         object
company_name              object
company_category_code     object
company_country_code      object
company_state_code        object
company_region            object
company_city              object
investor_permalink        object
investor_name             object
investor_country_code    float64
investor_state_code      float64
investor_region           o

In [107]:
# Analyze the unique value counts across all of the chunks to see if we can convert them to a numeric type.

chunk_iter = pd.read_csv( 'https://bit.ly/3BPcobU', encoding = "latin1",  usecols=use_cols, chunksize=5000)

unique_count = []
for chunk in chunk_iter:
  # Optimize numeric column
  chunk['raised_amount_usd'] = chunk['raised_amount_usd'].fillna(0)
  change_to_int(chunk, 'raised_amount_usd' )

  # Optimize 'funded_quarter' column
  chunk['funded_quarter'] = chunk['funded_quarter'].str.extract(r'Q(\d)', expand=False)
  chunk['funded_quarter'] = chunk['funded_quarter'].fillna(-1)
  chunk['funded_quarter'] = chunk['funded_quarter'].astype('int8')

  #determine the values count per columns. columns with less unique <50 can be converted to category data type
  chunk_unique_values = chunk.select_dtypes(include=['object']).nunique() #pd.series with unique values per col
  chunk_values_count = chunk.select_dtypes(include=['object']).count() #total no of items in col
  unique_count.append((100*chunk_unique_values/chunk_values_count)) #add each chunk series to a list

#concatenate the series in the list to a series
total_values_uniqueness = pd.concat(unique_count)
total_values_uniqueness = total_values_uniqueness.groupby(total_values_uniqueness.index).mean()
categorise_cols = list((total_values_uniqueness[total_values_uniqueness <50]).index)
print(f'unique values percentage:\n{total_values_uniqueness}')
print(f'columns to categorise:\n{categorise_cols}')

unique values percentage:
company_category_code     0.885323
company_city              9.884701
company_country_code      0.023168
company_name             57.691393
company_permalink        57.691393
company_region            3.899473
company_state_code        0.959790
funded_at                34.477606
funding_round_type        0.169454
investor_city             5.098421
investor_country_code     0.762508
investor_name            20.868260
investor_permalink       21.044168
investor_region           2.432371
investor_state_code       0.915164
dtype: float64
columns to categorise:
['company_category_code', 'company_city', 'company_country_code', 'company_region', 'company_state_code', 'funded_at', 'funding_round_type', 'investor_city', 'investor_country_code', 'investor_name', 'investor_permalink', 'investor_region', 'investor_state_code']


In [108]:
#optimize string cols 
total_memory = 0

chunk_iter = pd.read_csv( 'https://bit.ly/3BPcobU', encoding = "latin1",  usecols=use_cols, chunksize=5000)
for chunk in chunk_iter:
    # Optimize numeric column
    chunk['raised_amount_usd'] = chunk['raised_amount_usd'].fillna(0)
    change_to_int(chunk, 'raised_amount_usd' )

    # Optimize 'funded_quarter' column
    chunk['funded_quarter'] = chunk['funded_quarter'].str.extract(r'Q(\d)', expand=False)
    chunk['funded_quarter'] = chunk['funded_quarter'].fillna(-1)
    chunk['funded_quarter'] = chunk['funded_quarter'].astype('int8')

    #convert columns with few unique values to category dtype
    chunk[categorise_cols] = chunk[categorise_cols].astype('category')

    chunk_memory = chunk.memory_usage(deep=True).sum()
    total_memory += chunk_memory
    print('Chunk memory usage (MB) =', chunk_memory/(1024**2))

print('--'*20)
print('Total memory usage (MB) =', total_memory/(1024**2))

Chunk memory usage (MB) = 1.4321155548095703
Chunk memory usage (MB) = 1.253652572631836
Chunk memory usage (MB) = 1.2562618255615234
Chunk memory usage (MB) = 1.1977252960205078
Chunk memory usage (MB) = 1.2432384490966797
Chunk memory usage (MB) = 1.2561683654785156
Chunk memory usage (MB) = 1.2663860321044922
Chunk memory usage (MB) = 1.242548942565918
Chunk memory usage (MB) = 1.3025436401367188
Chunk memory usage (MB) = 1.3927536010742188
Chunk memory usage (MB) = 0.8324642181396484
----------------------------------------
Total memory usage (MB) = 13.675858497619629


We have achieved further 26MB memory savings by converting some string columns with `<50%` unique values to categorical. 

## Question 3: Loading Chunks Into SQLite

Now we're in good shape to start exploring and analyzing the data. The next step is to load each chunk into a table in a SQLite database so we can query the full data set.

**Tasks**

1. Create and connect to a new SQLite database file.
2. Expand on the existing chunk processing code to export each chunk to a new table in the SQLite database.
3. Query the table and make sure the data types match up to what you had in mind for each column.

In [148]:
# Your code goes here
import sqlite3

 # Create and connect to a new database
conn = sqlite3.connect('crunchbase.db')
cur = conn.cursor()
cur.execute('''DROP TABLE IF EXISTS investments''')



chunk_iter = pd.read_csv( 'https://bit.ly/3BPcobU', encoding = "latin1",  usecols=use_cols, chunksize=5000)
for chunk in chunk_iter:
     
     # Optimize numeric column
    chunk['raised_amount_usd'] = chunk['raised_amount_usd'].fillna(0)
    change_to_int(chunk, 'raised_amount_usd' )
    
    #drops rows where cols with null values
    chunk = chunk[chunk['raised_amount_usd'] !=0] 

    # Optimize 'funded_quarter' column
    chunk['funded_quarter'] = chunk['funded_quarter'].str.extract(r'Q(\d)', expand=False)
    chunk['funded_quarter'] = chunk['funded_quarter'].fillna(-1)
    chunk['funded_quarter'] = chunk['funded_quarter'].astype('int8')

    #convert columns with few unique values to category dtype
    chunk[categorise_cols] = chunk[categorise_cols].astype('category')

    #store the data in crunchbase databse
    chunk.to_sql('investments', conn, if_exists='append', index=False)

In [149]:
#querying the created table crunchbase from the database
verify_df = pd.read_sql('PRAGMA table_info(investments)', conn)
verify_df

,cid,name,type,notnull,dflt_value,pk
0,0,company_permalink,TEXT,0,None,0
1,1,company_name,TEXT,0,None,0
2,2,company_category_code,TEXT,0,None,0
3,3,company_country_code,TEXT,0,None,0
4,4,company_state_code,TEXT,0,None,0
5,5,company_region,TEXT,0,None,0
6,6,company_city,TEXT,0,None,0
7,7,investor_permalink,TEXT,0,None,0
8,8,investor_name,TEXT,0,None,0
9,9,investor_country_code,TEXT,0,None,0


In [150]:
verify_df = pd.read_sql('SELECT * FROM investments LIMIT 3', conn)
verify_df

,company_permalink,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_quarter,raised_amount_usd
0,/company/advercar,AdverCar,advertising,USA,CA,SF Bay,San Francisco,/company/1-800-flowers-com,1-800-FLOWERS.COM,USA,NY,New York,New York,series-a,2012-10-30,4,2000000
1,/company/launchgram,LaunchGram,news,USA,CA,SF Bay,Mountain View,/company/10xelerator,10Xelerator,USA,OH,Columbus,Columbus,other,2012-01-23,1,20000
2,/company/utap,uTaP,messaging,USA,None,United States - Other,None,/company/10xelerator,10Xelerator,USA,OH,Columbus,Columbus,other,2012-01-01,1,20000


## Question 4: Next Steps

Now that the data is in SQLite, we can use the pandas SQLite workflow we learned in the last lesson to explore and analyze startup investments. Remember that each row isn't a unique company, but a unique investment from a single investor. This means that many startups will span multiple rows.

Use the pandas SQLite workflow to answer the following questions:

* What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.
* Which category of company attracted the most investments?
* Which investor contributed the most money (across all startups)?
* Which investors contributed the most money per startup?
* Which funding round was the most popular? Which was the least popular?

Here are some ideas for further exploration:

* Repeat the tasks in this project using stricter memory constraints (under 1 megabyte).
* Clean and analyze the other Crunchbase data sets from the same GitHub repo.
* Understand which columns the data sets share, and how the data sets are linked.
* Create a relational database design that links the data sets together and reduces the overall disk space the database file consumes.

Use pandas to populate each table in the database, create the appropriate indexes, and so on.

In [151]:
query = '''
    SELECT 
        company_name, 
        company_category_code,
        investor_name,
        funding_round_type,
        raised_amount_usd
    FROM 
        investments
'''
results_df = pd.read_sql(query, conn)

grouped = results_df.groupby(by='company_name').raised_amount_usd.sum()     #get results by co name and  amt
ncompanies = results_df.nunique().company_name    #get unique Co's
ordered = grouped.sort_values(ascending=False)

**What proportion of the total amount of funds did the top 10%  and bottom 10% raise?** 

In [152]:
top_10pct = ordered.head(int(round(ncompanies*.10)))
bottom_10pct = ordered.tail(int(round(ncompanies*.10)))

print(f'\n Top 10% raised: {100*top_10pct.sum()/ grouped.sum()}%. In amount: {top_10pct.sum()} / {grouped.sum()}' )
print(f'\n Bottom 10% raised: {100*bottom_10pct.sum()/ grouped.sum()}%. In amount: {bottom_10pct.sum()} / {grouped.sum()}' )


 Top 10% raised: 64.56178900319603%. In amount: 440138496766 / 681732187973

 Bottom 10% raised: 0.02676334405486896%. In amount: 182454331 / 681732187973


**What proportion of the total amount of funds did the top 1%  and  bottom 1% raise?**

In [153]:
top_1pct = ordered.head(int(round(ncompanies*.01)))
bottom_1pct = ordered.tail(int(round(ncompanies*.01)))

print(f'\n Top 1% raised: {100*top_1pct.sum()/ grouped.sum()}%. In amount: {top_1pct.sum()} / {grouped.sum()}' )
print(f'\n Bottom 1% raised: {100*bottom_1pct.sum()/ grouped.sum()}%. In amount: {bottom_1pct.sum()} / {grouped.sum()}' )



 Top 1% raised: 25.14079032847251%. In amount: 171392859980 / 681732187973

 Bottom 1% raised: 0.00022018910453138984%. In amount: 1501100 / 681732187973


**Observations**



1.   Top 10% raised 64.5% of the total amount of funds
2.   Top 1% raised 25.1% of the total amount of funds
3.   Bottom 10% raised 0.02676% of the total amount of funds
4.   Bottom 1% raised 0.00022% of the total amount of funds



**Which category of company attracted the most investments?**

In [156]:
results_df.groupby(by='company_category_code').raised_amount_usd.sum().sort_values(ascending=False).head(1)

company_category_code
biotech    110396423062
Name: raised_amount_usd, dtype: int64

**Which investor contributed the most money (across all startups)?**

In [157]:
results_df.groupby(by='investor_name').raised_amount_usd.sum().sort_values(ascending=False).head(1)

investor_name
Kleiner Perkins Caufield & Byers    11217826376
Name: raised_amount_usd, dtype: int64

**Which investors contributed the most money per startup?**

In [158]:
sums = results_df.groupby(by=['company_name', 'investor_name']).raised_amount_usd.sum()
ranked = sums.groupby(by='company_name').rank(method='dense', ascending=False)
maxes = ranked[ranked == 1]
maxes

company_name  investor_name       
#waywire      First Round Capital     1.0
              Innovation Endeavors    1.0
              Jeff Weiner             1.0
              Oprah Winfrey           1.0
              Troy Carter             1.0
                                     ... 
ybuy          TomorrowVentures        1.0
zozi          500 Startups            1.0
              LaunchCapital           1.0
              PAR Capital Ventures    1.0
zulily        Andreessen Horowitz     1.0
Name: raised_amount_usd, Length: 24030, dtype: float64

**Which funding round was the most popular? Which was the least popular?**

In [160]:
# Which funding round was the most popular? Which was the least popular?

rounds = results_df.funding_round_type.value_counts()

# Most popular
print("Most Popular: ", rounds.idxmax())
print("least populat: ", rounds.idxmin())

Most Popular:  series-a
least populat:  crowdfunding
